# Page Template
This notebook contains commands and procedures necessary for all new pages.

v1.2.0

## Global Variables/Commands

In [1]:
# Imports
import templates
import re
import xml.etree.ElementTree as ET
import json
import os
import datetime
import pytz

In [ ]:
# Variables
#target_domain

### Action Template

In [ ]:
# Action Template

#force_run = False

#cmd_cache = ''
#cmd_syntax = ''
#cmd_os = ''

#output = templates.cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

#print(output)

## OSINT Recon

In [ ]:
# Whois
#TODO Use requests to query whois site: https://whois.arin.net/ui/
#TODO Add info recieved into 'loot' directory or page

### Organizational Data
- Organizational Structure
  - Subordinate, senior and lateral organizations
  - Subsidaries
  - Public Presence
  - Organizational Information
- Organizational IT Data
  - Repository commit history
  - Email addresses or email address naming convention
  - Files containing sensitive data and/or metadata - version numbers, applications, process/procedures, etc.
- Data in External Sources
  - Financials that indicate IT spending (security software purchases, etc.)
  - Personnel on social media
  - Customer technologies in resumes, newsgroups, LinkedIn skills, etc.

**Add to Loot**
- Company mission
- Current events related to the company - acquisitions, conferences, etc.
- Points of contact
- Blogs (company and employee's personal)
- Backend infrastructure
- Financial and business information
- Job descriptions for IT positions
- Vendor names

### Github/Pastebin Scraping
- [Truffle Hog](https://github.com/trufflesecurity/trufflehog)
- [Git Harvester](https://github.com/metac0rtex/GitHarvester)
- [TrashPanda - OSINT Bot](https://got-hacked.wtf/)

### NMAP Scans
_Reference: [NMAP Cheat Sheet](https://www.stationx.net/nmap-cheat-sheet/)_

#### NMAP Basic Alive Scans

##### Command Switches
- `-n` No DNS resolution
- `-sn` Disable port scanning
- `-oA` Output in .nmap, greppable nmap (.gnmap), and .xml

In [ ]:
# NMAP Basic Alive Scan
#TODO: Only supports scanning one subnet at a time currently
force_run = False
target_subnet = '192.168.0.0'
target_subnet_class = '24'
cmd_cache = f'nmap_alive_{target_subnet}_{target_subnet_class}'
cmd_syntax = f'nmap -sn -n -oA {templates.var_vm_lootdir}/nmap/{target_subnet}_{target_subnet_class}_alive_basic {target_subnet}/{target_subnet_class}'
#NOTE: Will only reliably accept 'vm'
cmd_os = 'vm'

output = templates.cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

# Extract alive IPs
alives_ips = re.findall(r'Nmap scan report for (\S+)', output)
alives_ips = [ip for ip in alives_ips if ip != 'Host']

# Save alives to Loot
with open(f'Loot/nmap/{target_subnet}_{target_subnet_class}_alive_basic', 'w') as file:
    for ip in alives_ips:
        file.write(ip + '\n')
print(f'Alive IPs for {target_subnet}/{target_subnet_class} have been written to Loot/nmap/')

# Save alives to VM
templates.command_scp_put_vm(f'Loot/nmap/{target_subnet}_{target_subnet_class}_alive', f'{templates.var_vm_lootdir}/nmap/{target_subnet}_{target_subnet_class}_alive_basic')

#### NMAP Robust Alive Scans

_Note: This is an aggressive scan which will probe all specified ports on every device in subnet_

##### Command Switches
- `-T3` Timing (default speed)
- `--reason` Display the reason a port is in a particular state
- `--randomize hosts` Scan hosts in random order
- `-v` Increase verbosity (1x)
- `-n` No DNS resolution
- `-PE` ICMP echo requests to target (ping)
- `-PM` ICMP timestamp requests
- `-PO` Ping protocol discovery (port 1)
- `-PU` UDP ping discovery (default port 40125)
- `-PS` TCP SYN discovery on specified ports
- `-PA` TCP ACK discovery on specified ports
- `-sS` TCP SYN port scan (default)
- `-sV` Attempts to determine the version
- `-oA` Output in .nmap, greppable nmap (.gnmap), and .xml

_Note: currently the robust scan is double up on ports in `-sS` and `-PS` unsure if sending double requests but could remove for clarity_

In [6]:
# NMAP Robust Alive Scan
#TODO: Only supports scanning one subnet at a time currently
#TODO: Kind of scary that running this again against the same subnet would overwrite the old info, might not matter though
#NOTE: This scan requires root privileges
force_run = False

target_subnet = '192.168.0.0'
target_subnet_class = '24'
cmd_cache = f'nmap_robust_alive_{target_subnet}_{target_subnet_class}'
cmd_syntax = f'sudo nmap -T3 --log-errors --reason --randomize-hosts -v -n -PE -PM -PO -PU -PS80,23,443,21,22,25,3389,110,445,139 -PA80,443,22,445,129 -sS -sV -p21,22,23,25,80,443,8080,8443 -oA {templates.var_vm_lootdir}/nmap/{target_subnet}_{target_subnet_class}_alive_robust {target_subnet}/{target_subnet_class}'
#NOTE: Will only reliably accept 'vm'
cmd_os = 'vm'

output = templates.cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)
print('[+] Finished NMAP scan')
#print(output)

# Extract alive IPs and attributes from .xml
cmd_cache = f'store_nmap_robust_alive_{target_subnet}_{target_subnet_class}'
cmd_syntax = f'cat {templates.var_vm_lootdir}/nmap/{target_subnet}_{target_subnet_class}_alive_robust.xml'

xml_output = templates.cache_command_output(cmd_cache, force_run, cmd_os, cmd_syntax)

with open('Loot/nmap/tmp_xml', 'w') as file:
    file.write(xml_output)

# Parse the XML file
tree = ET.parse('Loot/nmap/tmp_xml')
root = tree.getroot()

# Remove temporary .xml
if os.path.exists("Loot/nmap/tmp_xml"):
  os.remove("Loot/nmap/tmp_xml")
else:
  print("The temporary .xml file does not exist")
  
# Initialize a dictionary to store the parsed data
nmap_data = {"hosts": []}

# Establish time conversion for nmap output
#NOTE: Configured to output in Pacific time
def convert_epoch_pst(epoch_time):
    epoch_time = int(epoch_time)
    timestamp_utc = datetime.datetime.utcfromtimestamp(epoch_time)
    pst_timezone = pytz.timezone('America/Los_Angeles')
    timestamp_pst = timestamp_utc.replace(tzinfo=pytz.utc).astimezone(pst_timezone)
    formatted_timestamp = timestamp_pst.strftime('%H:%M:%S_%Z')
    return formatted_timestamp


# Parse each host in the XML
for host in root.findall('.//host'):
    # Extracting basic host information
    host_attribs = host.attrib
    status = host.find('status').attrib
    address_info = host.find('address').attrib

    host_entry = {
        "ip": address_info['addr'],
        "status": status['state'],
        "reason": status['reason'],
        "scan_starttime": convert_epoch_pst(host_attribs['starttime']),
        "scan_endtime": convert_epoch_pst(host_attribs['endtime'])
    }

    # Extracting port information
    ports = []
    for port in host.findall('.//port'):
        port_data = {
            "portid": port.attrib['portid'],
            "protocol": port.attrib['protocol'],
            "state": port.find('state').attrib['state'],
            "service": port.find('service').attrib['name'] if port.find('service') is not None else None
        }
        ports.append(port_data)

    host_entry['ports'] = ports
    nmap_data['hosts'].append(host_entry)

# Sort NMAP data by IP and convert the dictionary to a JSON string
nmap_data['hosts'] = sorted(nmap_data['hosts'], key=lambda x: tuple(int(part) for part in x['ip'].split('.')))
nmap_json = json.dumps(nmap_data, indent=4)

# Configure attack surface list
#NOTE: This assumes the reset on the host means it is unreachable (most likely because firewall or not online)
attack_surface_dict = []
for host in nmap_data['hosts']:
    if host['reason'] != 'reset':
        for port in host['ports']:
            attack_surface_dict.append(f"{host['ip']}: {port['portid']}")

# Save the robust alive scan JSON data to a file
json_file = f'Loot/nmap/{target_subnet}_{target_subnet_class}_alive_robust.json'
with open(json_file, 'w') as file:
    file.write(nmap_json)
print(f'[+] JSON Scan output written to {json_file}')

# Save the attack surface to a file
attack_surface_file = f'Loot/nmap/{target_subnet}_{target_subnet_class}_attack_surface'
with open(attack_surface_file, 'w') as file:
    for entry in attack_surface_dict:
        file.write(f'{entry}\n')
print(f'[+] Assumed attack surface written to {attack_surface_file}')

[*] Loading data from cache.
[+] Finished NMAP scan
[*] Loading data from cache.
[+] JSON Scan output written to Loot/nmap/192.168.0.0_24_robust_alive.json
[+] Assumed attack surface written to Loot/nmap/192.168.0.0_24_attack_surface


#### NMAP Service Scan

In [ ]:
#!/bin/sh
# echo "FTP scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p21 -oA ad.ftp
# echo "ssh scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p22 -oA ad.ssh
# echo "telnet scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p23 -oA ad.telnet
# echo "SMB scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p135,139,445 -oA ad.smb
# echo "RDS / Terminal services scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p3389 -oA ad.rds
# echo "IPMI TCP scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p623 -oA ad.ipmi_tcp
# echo "web quick scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p80,8080,443,8443 -oA ad.web_quick
# echo "mssql scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p1433 -oA ad.mssql
# echo "mysql scan" 
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p3306 -oA ad.mysql
# echo "VNC scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p5800,5900-5910,6000 -oA ad.vnc
# echo "X11 scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p6000-6063 -oA ad.x11
# echo "smart install"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p4786 -oA ad.cisco_smartinstall
# echo "Redis scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p6379 -oA ad.redis
# echo "mongodb scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p27017 -oA ad.mongodb
# echo "java RMI"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p1090,1098,1099,4444,11099,47001,47002,10999 -oA ad.java_RMI
# echo "weblogic"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p7000-7004,8000-8003,9000-9003,9503,7070,7071 -oA ad.weblogic
# echo "JDWP"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p45000,45001 -oA ad.JDWP
# echo "JMX"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p8686,9012,50500 -oA ad.JMX
# echo "GlassFish scan"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p4848 -oA ad.glassfish
# echo "jboss"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p11111,4444,4445 -oA ad.jboss
# echo "HP data protector"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p5555,5556 -oA ad.hp_data_protector
# echo "rexecd"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p512 -oA ad.rexecd
# echo "SNMP"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p161 -sU -oA ad.snmp
# echo "IPMI UDP"
# nmap -iL ad.ping_live.txt -vv -T4 --open -sV -Pn -n -p623 -sU -oA ad.ipmi_udp